In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torchvision.models import vgg16
import time
import copy

In [ ]:
# Kiểm tra và sử dụng GPU nếu có
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def load_cifar10():
    transform = transforms.Compose([
        transforms.Resize(254),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    subset_size = 1000
    subset_indices = torch.randperm(len(trainset))[:subset_size]
    subset = torch.utils.data.Subset(trainset, subset_indices)
    trainset = subset
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
    return trainset, trainloader

trainset, trainloader = load_cifar10()


print("\nCIFAR-10 Dataset Information:")
print(f"Number of training samples: {len(trainset)}")
print(f"Number of batches: {len(trainloader)}")
print(f"Batch size: {trainloader.batch_size}")

# Result #
Files already downloaded and verified

CIFAR-10 Dataset Information:
Number of training samples: 1000
Number of batches: 32
Batch size: 32


Files already downloaded and verified

CIFAR-10 Dataset Information:
Number of training samples: 1000
Number of batches: 32
Batch size: 32


In [ ]:
def load_vgg16():
    model = vgg16(weights=None)
    model.classifier[6] = nn.Linear(4096, 10)  # Thay đổi lớp cuối cùng cho CIFAR-10
    model = model.to(device)
    return model

model = load_vgg16()

print("VGG16 Model Structure:")
print(model)

VGG16 Model Structure:
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2

In [ ]:
# Hàm để tính số lượng tham số của mô hình
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Số lượng tham số của mô hình ban đầu: {count_parameters(model)}")

#Result
Số lượng tham số của mô hình ban đầu: 134301486

Số lượng tham số của mô hình ban đầu: 134301486


In [ ]:
# 2. Thực hiện prune một filter và xem sự thay đổi về shape
def prune_filter(model, layer_index, filter_index):
    conv_layer = model.features[layer_index]
    next_conv_layer = None
    
    # Tìm lớp tích chập tiếp theo
    for layer in model.features[layer_index+1:]:
        if isinstance(layer, nn.Conv2d):
            next_conv_layer = layer
            break

    # Tạo một lớp tích chập mới với số filter giảm đi 1
    new_conv = nn.Conv2d(in_channels=conv_layer.in_channels,
                         out_channels=conv_layer.out_channels - 1,
                         kernel_size=conv_layer.kernel_size,
                         stride=conv_layer.stride,
                         padding=conv_layer.padding,
                         dilation=conv_layer.dilation,
                         groups=conv_layer.groups,
                         bias=conv_layer.bias is not None)

    # Sao chép trọng số và bias, ngoại trừ filter được prune
    new_filters = torch.cat((conv_layer.weight.data[:filter_index], conv_layer.weight.data[filter_index+1:]))
    new_conv.weight.data = new_filters

    if conv_layer.bias is not None:
        new_biases = torch.cat((conv_layer.bias.data[:filter_index], conv_layer.bias.data[filter_index+1:]))
        new_conv.bias.data = new_biases

    # Thay thế lớp tích chập cũ bằng lớp mới trong mô hình
    model.features[layer_index] = new_conv

    # Điều chỉnh lớp tích chập tiếp theo nếu có
    if next_conv_layer is not None:
        next_new_conv = nn.Conv2d(in_channels=next_conv_layer.in_channels - 1,
                                  out_channels=next_conv_layer.out_channels,
                                  kernel_size=next_conv_layer.kernel_size,
                                  stride=next_conv_layer.stride,
                                  padding=next_conv_layer.padding,
                                  dilation=next_conv_layer.dilation,
                                  groups=next_conv_layer.groups,
                                  bias=next_conv_layer.bias is not None)

        next_new_conv.weight.data = next_conv_layer.weight.data[:, :filter_index, :, :].clone()
        next_new_conv.weight.data = torch.cat([next_new_conv.weight.data, next_conv_layer.weight.data[:, filter_index+1:, :, :]], dim=1)

        if next_conv_layer.bias is not None:
            next_new_conv.bias.data = next_conv_layer.bias.data.clone()

        # Tìm index của lớp tích chập tiếp theo
        for i, layer in enumerate(model.features[layer_index+1:]):
            if isinstance(layer, nn.Conv2d):
                next_layer_index = layer_index + 1 + i
                break

        model.features[next_layer_index] = next_new_conv

    return model

In [ ]:
# Prune filter đầu tiên của lớp tích chập đầu tiên
pruned_model = prune_filter(copy.deepcopy(model), 0, 0)

print("\nShape của lớp tích chập đầu tiên trước khi pruning:")
print(model.features[0].weight.shape)
print("Shape của lớp tích chập đầu tiên sau khi pruning:")
print(pruned_model.features[0].weight.shape)

print(f"\nSố lượng tham số sau khi pruning: {count_parameters(pruned_model)}")


Shape của lớp tích chập đầu tiên trước khi pruning:
torch.Size([64, 3, 3, 3])
Shape của lớp tích chập đầu tiên sau khi pruning:
torch.Size([63, 3, 3, 3])

Số lượng tham số sau khi pruning: 138356940


In [ ]:
# Hàm để tái cấu trúc mô hình
def restructure_model(model, layer_index):
    next_conv_layer = None
    for i in range(layer_index + 1, len(model.features)):
        if isinstance(model.features[i], nn.Conv2d):
            next_conv_layer = model.features[i]
            break

    if next_conv_layer is not None:
        new_in_channels = next_conv_layer.in_channels - 1
        new_conv = nn.Conv2d(new_in_channels, next_conv_layer.out_channels,
                             next_conv_layer.kernel_size, next_conv_layer.stride,
                             next_conv_layer.padding, next_conv_layer.dilation,
                             next_conv_layer.groups, bias=next_conv_layer.bias is not None)

        # Sao chép trọng số, ngoại trừ kênh đầu vào bị loại bỏ
        new_weights = next_conv_layer.weight.data[:, torch.arange(next_conv_layer.in_channels) != filter_index]
        new_conv.weight.data = new_weights

        if next_conv_layer.bias is not None:
            new_conv.bias.data = next_conv_layer.bias.data

        model.features[i] = new_conv

    return model

In [ ]:
# Hàm để tái cấu trúc mô hình
def restructure_model(model, layer_index, filter_index):
    for i in range(layer_index + 1, len(model.features)):
        if isinstance(model.features[i], nn.Conv2d):
            conv_layer = model.features[i]
            new_in_channels = conv_layer.in_channels - 1
            new_conv = nn.Conv2d(new_in_channels, conv_layer.out_channels,
                                 conv_layer.kernel_size, conv_layer.stride,
                                 conv_layer.padding, conv_layer.dilation,
                                 conv_layer.groups, bias=conv_layer.bias is not None)
            
            # Sao chép trọng số, ngoại trừ kênh đầu vào bị loại bỏ
            new_weights = conv_layer.weight.data[:, torch.arange(conv_layer.in_channels) != filter_index]
            new_conv.weight.data = new_weights
            
            if conv_layer.bias is not None:
                new_conv.bias.data = conv_layer.bias.data
            
            model.features[i] = new_conv
        elif isinstance(model.features[i], nn.BatchNorm2d):
            bn_layer = model.features[i]
            new_bn = nn.BatchNorm2d(bn_layer.num_features - 1)
            
            # Sao chép các tham số, ngoại trừ feature bị loại bỏ
            new_bn.weight.data = bn_layer.weight.data[torch.arange(bn_layer.num_features) != filter_index]
            new_bn.bias.data = bn_layer.bias.data[torch.arange(bn_layer.num_features) != filter_index]
            new_bn.running_mean = bn_layer.running_mean[torch.arange(bn_layer.num_features) != filter_index]
            new_bn.running_var = bn_layer.running_var[torch.arange(bn_layer.num_features) != filter_index]
            
            model.features[i] = new_bn
    
    return model


In [ ]:
# Hàm để huấn luyện mô hình
def train_model(model, trainloader, epochs=5):
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

  for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    print('-' * 10)

    model.train
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
      inputs, labels = data[0].to(device), data[1].to(device)

      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item() * inputs.size(0)


      # epoch_loss = running_loss / len(trainloader.dataset)
      # print(f'Training Loss: {epoch_loss:.4f}')



      if i % 100 == 99:
        print(f'[{epoch + 1}, {i + 1:5d}] | Training loss: {running_loss / 100:.3f}')
        running_loss = 0.0
    print('Finished Training')
    return model, loss.item()

In [ ]:
# Hàm tổng quát để prune một filter bất kỳ và tái cấu trúc mô hình
def prune_and_restructure(model, layer_index, filter_index):
    model = prune_filter(model, layer_index, filter_index)
    model = restructure_model(model, layer_index)
    return model

In [ ]:
print("\nTraining mô hình gốc...")
start_time = time.time()
_, original_loss = train_model(model, trainloader)
end_time = time.time()
print(f"Thời gian training mô hình gốc: {end_time - start_time:.2f} giây")
print(f"Loss cuối cùng của mô hình gốc: {original_loss:.4f}")


Training mô hình gốc...
Epoch 1/5
----------
Finished Training
Thời gian training mô hình gốc: 2165.56 giây
Loss cuối cùng của mô hình gốc: 4.8297


In [ ]:
# Prune một filter và xem sự thay đổi
layer_index = 0  # Ví dụ: prune filter đầu tiên của lớp conv đầu tiên
filter_index = 0
model = prune_filter(model, layer_index, filter_index)

print("\nModel structure after pruning:")
print(model)

# Huấn luyện mô hình sau khi prune
print("\nTraining mô hình đã pruning...")
start_time = time.time()
_, pruned_loss = train_model(pruned_model, trainloader, epochs = 1)

end_time = time.time()
print(f"Thời gian training mô hình đã pruning: {end_time - start_time:.2f} giây")
print(f"Loss cuối cùng của mô hình đã pruning: {pruned_loss:.4f}")

print(f"\nChênh lệch loss: {abs(original_loss - pruned_loss):.4f}")

# Tái cấu trúc mô hình
model = restructure_model(model, layer_index)

print("\nModel structure after restructuring:")
print(model)

In [ ]:
# Huấn luyện mô hình sau khi tái cấu trúc
print("\nTraining mô hình đã tái cấu trúc...")
start_time = time.time()
_, restruct_loss = train_model(model, trainloader, epochs=1)

end_time = time.time()
print(f"Thời gian training mô hình đã pruning: {end_time - start_time:.2f} giây")
print(f"Loss cuối cùng của mô hình đã pruning: {restruct_loss:.4f}")

print(f"\nChênh lệch loss: {abs(original_loss - restruct_loss):.4f}")

In [ ]:
# Sử dụng hàm tổng quát để prune và tái cấu trúc
layer_index = 2  # Ví dụ: prune filter của lớp conv thứ 3
filter_index = 1
model = prune_and_restructure(model, layer_index, filter_index)

print("\nModel structure after pruning and restructuring using general function:")
print(model)

# Huấn luyện mô hình cuối cùng
print("\nTraining mô hình...")
start_time = time.time()
_, restruct_loss = train_model(model, trainloader, epochs=1)

end_time = time.time()
print(f"Thời gian training mô hình đã pruning: {end_time - start_time:.2f} giây")
print(f"Loss cuối cùng của mô hình đã pruning: {restruct_loss:.4f}")

print(f"\nChênh lệch loss: {abs(original_loss - restruct_loss):.4f}")